# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**: Structured Streaming with Files Description

**Fecha**:28/03/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Structured-Streaming") \
    .master("spark://b631371550f9:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/02 21:07:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Checa el pwd

In [4]:
!pwd

/home/jovyan


### Generate logs

In [5]:
from team_bellaco.log_gen import generate_logs
generate_logs(100,"/home/jovyan/notebooks/data/logs","log")

Logs guardados en: /home/jovyan/notebooks/data/logs/log-99.log


Registra el Listener del Team Bellaco

In [6]:
from team_bellaco.traffic_query_listener import TrafficListener
spark.streams.addListener(TrafficListener())

Lee los logs como stream

In [7]:
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import split, col

log_schema = StructType().add("raw", StringType())

log_df = (
    spark.readStream.format("text")
    .schema(log_schema)
    .load("/home/jovyan/notebooks/data/logs/")
)

Separa las columnas de los logs (timestamp, level, message, server)

In [8]:
parsed_df = (
    log_df.withColumn("parts", split(col("raw"), " \\| "))
    .withColumn("timestamp", col("parts").getItem(0))
    .withColumn("level", col("parts").getItem(1))
    .withColumn("message", col("parts").getItem(2))
    .withColumn("server", col("parts").getItem(3))
    .drop("parts")
)

Filtra los errores 500 de los logs

In [9]:
errors_df = parsed_df.filter(col("message").contains("500 Internal Server Error"))

Opcion 1: Se ejecuta el stream y se detiene con el tiempo agregado en segundos

In [ ]:
import time

query = (
    errors_df.writeStream.outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

time.sleep(30) #Aqui son 30 Segundos
query.stop()

25/04/02 21:32:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a22bd96d-f4c6-46b7-ad8f-1029a9c57745. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/02 21:32:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: 2107f4c7-a2e8-429b-9b33-c84e61119305


25/04/02 21:32:47 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6397 milliseconds


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|raw                                                                    |timestamp          |level|message                  |server       |
+-----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-1|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-1|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-3|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-3|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-2|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-2|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | serv

Opcion 2: Se ejecuta el stream y se detiene manualmente como con el profe

In [10]:
query = (
    errors_df.writeStream.outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")
    .start()
)

query.awaitTermination()

25/04/02 21:32:15 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c0a406d5-c0e3-4b67-9875-b99b26654697. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/02 21:32:15 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: 3e89e315-e208-4412-98b2-758436394170


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|raw                                                                    |timestamp          |level|message                  |server       |
+-----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-1|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-1|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-3|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-3|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | server-node-2|2025-03-31 22:20:56|ERROR|500 Internal Server Error|server-node-2|
|2025-03-31 22:20:56 | ERROR | 500 Internal Server Error | serv

25/04/02 21:32:25 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 9908 milliseconds


Query made progress: {
  "id" : "3e89e315-e208-4412-98b2-758436394170",
  "runId" : "a2f18957-0770-4c3c-b287-feee8221ddf0",
  "name" : null,
  "timestamp" : "2025-04-02T21:32:15.671Z",
  "batchId" : 0,
  "numInputRows" : 20000,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 2023.8818053025705,
  "durationMs" : {
    "addBatch" : 4409,
    "commitOffsets" : 28,
    "getBatch" : 4411,
    "latestOffset" : 521,
    "queryPlanning" : 452,
    "triggerExecution" : 9881,
    "walCommit" : 36
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[file:/home/jovyan/notebooks/data/logs]",
    "startOffset" : null,
    "endOffset" : {
      "logOffset" : 0
    },
    "latestOffset" : null,
    "numInputRows" : 20000,
    "inputRowsPerSecond" : 0.0,
    "processedRowsPerSecond" : 2023.8818053025705
  } ],
  "sink" : {
    "description" : "org.apache.spark.sql.execution.streaming.ConsoleTable$@418cda46",
    "numOutputRows" : 1645
  }
}
Rows processed in

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
# Stop the SparkContext
sc.stop()